## Roughwork for Problem 2

This section is just to help me understand what the code is doing.

Here I:
- Look at the cube root of individual primes.
- Inspect fractional parts.
- Verify one or two constants manually.


In [9]:
# Exploring how the constant is derived for the prime p = 2.

import numpy as np


p = 2

# Compute the cube root (as float).
cbrt_p = np.cbrt(np.float64(p))
print("cuberoot(2) =", cbrt_p)

# Fractional part of the cube root.
frac_p = cbrt_p - np.floor(cbrt_p)
print("fractional part =", frac_p)

# Multiply fractional part by 2**32.
scaled_p = frac_p * (2**32)
print("frac * 2**32 =", scaled_p)

# Take integer part and show in decimal and hex.
bits_p = int(scaled_p)
print("integer bits =", bits_p)
print("hex constant =", f"0x{bits_p:08x}")


cuberoot(2) = 1.259921049894873
fractional part = 0.25992104989487297
frac * 2**32 = 1116352408.8404636
integer bits = 1116352408
hex constant = 0x428a2f98


In [10]:
def debug_constant_for_prime(p):
    """
    Print all intermediate steps for the constant derived from a prime p.
    This is just for my own understanding and rough checking.
    """
    x = np.float64(p)
    cbrt = np.cbrt(x)
    floor_cbrt = np.floor(cbrt)
    frac = cbrt - floor_cbrt
    scaled = frac * (2**32)
    bits = int(scaled)

    print(f"Prime p              = {p}")
    print(f"cuberoot(p)          = {cbrt}")
    print(f"floor(cuberoot(p))   = {floor_cbrt}")
    print(f"fractional part      = {frac}")
    print(f"fractional * 2**32   = {scaled}")
    print(f"integer bits         = {bits}")
    print(f"hex constant         = 0x{bits:08x}")
    print("-" * 40)


# Try this for a few of the first primes.
for p in [2, 3, 5]:
    debug_constant_for_prime(p)


Prime p              = 2
cuberoot(p)          = 1.259921049894873
floor(cuberoot(p))   = 1.0
fractional part      = 0.25992104989487297
fractional * 2**32   = 1116352408.8404636
integer bits         = 1116352408
hex constant         = 0x428a2f98
----------------------------------------
Prime p              = 3
cuberoot(p)          = 1.4422495703074083
floor(cuberoot(p))   = 1.0
fractional part      = 0.4422495703074083
fractional * 2**32   = 1899447441.1403713
integer bits         = 1899447441
hex constant         = 0x71374491
----------------------------------------
Prime p              = 5
cuberoot(p)          = 1.709975946676697
floor(cuberoot(p))   = 1.0
fractional part      = 0.7099759466766971
fractional * 2**32   = 3049323471.9230537
integer bits         = 3049323471
hex constant         = 0xb5c0fbcf
----------------------------------------


## Roughwork: Problem 3 Padding

In [11]:
msg = b"abc"
print("Original message:", msg)
print("Original bytes:  ", msg)
print("Original hex:    ", msg.hex())
print("Original bit len:", len(msg) * 8)

# Build padded message step by step (similar to block_parse but not as a generator).

bit_len = len(msg) * 8
padded = bytearray(msg)

# Append 0x80 (the '1' bit followed by seven 0 bits).
padded.append(0x80)
print("\nAfter appending 0x80 (hex):")
print(padded.hex())

# Append 0x00 until length ≡ 56 mod 64.
while (len(padded) % 64) != 56:
    padded.append(0x00)

print("\nAfter zero padding (hex):")
print(padded.hex())
print("Length before length-field (bytes):", len(padded))

# Append 64-bit big-endian length.
padded += bit_len.to_bytes(8, "big")

print("\nFinal padded message (hex):")
print(padded.hex())
print("Final padded length (bytes):", len(padded))
print("Final padded length % 64:", len(padded) % 64)

# Show last 8 bytes explicitly.
print("\nLast 8 bytes (hex):", padded[-8:].hex())
print("Decoded bit length:", int.from_bytes(padded[-8:], "big"))


Original message: b'abc'
Original bytes:   b'abc'
Original hex:     616263
Original bit len: 24

After appending 0x80 (hex):
61626380

After zero padding (hex):
6162638000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
Length before length-field (bytes): 56

Final padded message (hex):
61626380000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000018
Final padded length (bytes): 64
Final padded length % 64: 0

Last 8 bytes (hex): 0000000000000018
Decoded bit length: 24


## Roughwork: Problem 4 internals

Here I look inside one SHA-256 block processing


In [12]:
import numpy as np

# ==== helper bit functions from Problem 1 ====


def rotr(x, n):
    # rotate 32-bit value x right by n bits
    x = np.uint32(x)
    n = n % 32
    return np.uint32((x >> n) | (x << (32 - n)))


def shr(x, n):
    # logical right shift of x by n bits
    x = np.uint32(x)
    return np.uint32(x >> n)


def Ch(x, y, z):
    # SHA-256 choice function
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    return np.uint32((x & y) ^ (~x & z))


def Maj(x, y, z):
    # SHA-256 majority function
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    return np.uint32((x & y) ^ (x & z) ^ (y & z))


def Sigma0(x):
    # big Σ0 for SHA-256
    x = np.uint32(x)
    return np.uint32(rotr(x, 2) ^ rotr(x, 13) ^ rotr(x, 22))


def Sigma1(x):
    # big Σ1 for SHA-256
    x = np.uint32(x)
    return np.uint32(rotr(x, 6) ^ rotr(x, 11) ^ rotr(x, 25))


def sigma0(x):
    # small σ0 for SHA-256
    x = np.uint32(x)
    return np.uint32(rotr(x, 7) ^ rotr(x, 18) ^ shr(x, 3))


def sigma1(x):
    # small σ1 for SHA-256
    x = np.uint32(x)
    return np.uint32(rotr(x, 17) ^ rotr(x, 19) ^ shr(x, 10))


# ==== SHA-256 constants and initial hash ====


K_hex = [
    "428a2f98", "71374491", "b5c0fbcf", "e9b5dba5",
    "3956c25b", "59f111f1", "923f82a4", "ab1c5ed5",
    "d807aa98", "12835b01", "243185be", "550c7dc3",
    "72be5d74", "80deb1fe", "9bdc06a7", "c19bf174",
    "e49b69c1", "efbe4786", "0fc19dc6", "240ca1cc",
    "2de92c6f", "4a7484aa", "5cb0a9dc", "76f988da",
    "983e5152", "a831c66d", "b00327c8", "bf597fc7",
    "c6e00bf3", "d5a79147", "06ca6351", "14292967",
    "27b70a85", "2e1b2138", "4d2c6dfc", "53380d13",
    "650a7354", "766a0abb", "81c2c92e", "92722c85",
    "a2bfe8a1", "a81a664b", "c24b8b70", "c76c51a3",
    "d192e819", "d6990624", "f40e3585", "106aa070",
    "19a4c116", "1e376c08", "2748774c", "34b0bcb5",
    "391c0cb3", "4ed8aa4a", "5b9cca4f", "682e6ff3",
    "748f82ee", "78a5636f", "84c87814", "8cc70208",
    "90befffa", "a4506ceb", "bef9a3f7", "c67178f2",
]

K = [np.uint32(int(x, 16)) for x in K_hex]

H0_hex = [
    "6a09e667", "bb67ae85", "3c6ef372", "a54ff53a",
    "510e527f", "9b05688c", "1f83d9ab", "5be0cd19",
]

H0_init = [np.uint32(int(x, 16)) for x in H0_hex]


# ==== block_parse from Problem 3 ====


def block_parse(msg):
    # generator that yields padded 64 byte blocks from msg
    if not isinstance(msg, (bytes, bytearray)):
        raise TypeError("msg must be bytes or bytearray")

    bit_len = len(msg) * 8

    padded = bytearray(msg)         # start with original bytes
    padded.append(0x80)             # append 1 bit (10000000)

    # pad with zeros until length ≡ 56 (mod 64)
    while len(padded) % 64 != 56:
        padded.append(0x00)

    # append original bit length as 64 bit big endian
    padded += bit_len.to_bytes(8, "big")

    # yield 64 byte blocks
    for i in range(0, len(padded), 64):
        yield bytes(padded[i:i + 64])


# ==== message schedule builder from Problem 4 ====


def build_message_schedule(block):
    # build W[0..63] from a 64 byte block
    if len(block) != 64:
        raise ValueError("block must be exactly 64 bytes")

    W = [np.uint32(0) for _ in range(64)]

    # first 16 words from block as big endian 32 bit ints
    for i in range(16):
        start = i * 4
        word_bytes = block[start:start + 4]
        W[i] = np.uint32(int.from_bytes(word_bytes, "big"))

    # extend 16 to 64
    for t in range(16, 64):
        s0 = sigma0(W[t - 15])
        s1 = sigma1(W[t - 2])
        W[t] = np.uint32(W[t - 16] + W[t - 7] + s0 + s1)

    return W


In [13]:
test_msg = b"abc"

# Get padded blocks from block_parse
blocks = list(block_parse(test_msg))
first_block = blocks[0]

print("First block length:", len(first_block))
print("First block hex:")
print(first_block.hex())

# Build message schedule for this block
W_debug = build_message_schedule(first_block)

print("\nFirst 16 W[t] values:")
for t in range(16):
    print(f"W[{t:2d}] = 0x{int(W_debug[t]):08x}")

print("\nNext few W[t] values (t = 16..19):")
for t in range(16, 20):
    print(f"W[{t:2d}] = 0x{int(W_debug[t]):08x}")


First block length: 64
First block hex:
61626380000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000018

First 16 W[t] values:
W[ 0] = 0x61626380
W[ 1] = 0x00000000
W[ 2] = 0x00000000
W[ 3] = 0x00000000
W[ 4] = 0x00000000
W[ 5] = 0x00000000
W[ 6] = 0x00000000
W[ 7] = 0x00000000
W[ 8] = 0x00000000
W[ 9] = 0x00000000
W[10] = 0x00000000
W[11] = 0x00000000
W[12] = 0x00000000
W[13] = 0x00000000
W[14] = 0x00000000
W[15] = 0x00000018

Next few W[t] values (t = 16..19):
W[16] = 0x61626380
W[17] = 0x000f0000
W[18] = 0x7da86405
W[19] = 0x600003c6


C:\Users\natha\AppData\Local\Temp\ipykernel_19112\2986050169.py:136: RuntimeWarning: overflow encountered in scalar add
  W[t] = np.uint32(W[t - 16] + W[t - 7] + s0 + s1)


In [14]:
# Start from initial state for SHA-256
current = [np.uint32(x) for x in H0_init]
a = np.uint32(current[0])
b = np.uint32(current[1])
c = np.uint32(current[2])
d = np.uint32(current[3])
e = np.uint32(current[4])
f = np.uint32(current[5])
g = np.uint32(current[6])
h = np.uint32(current[7])

W0 = build_message_schedule(first_block)

t = 0
temp1 = np.uint32(
    h
    + Sigma1(e)
    + Ch(e, f, g)
    + K[t]
    + W0[t]
)
temp2 = np.uint32(
    Sigma0(a)
    + Maj(a, b, c)
)

print("Round t = 0 internal values")
print(f"a = 0x{int(a):08x}")
print(f"b = 0x{int(b):08x}")
print(f"c = 0x{int(c):08x}")
print(f"d = 0x{int(d):08x}")
print(f"e = 0x{int(e):08x}")
print(f"f = 0x{int(f):08x}")
print(f"g = 0x{int(g):08x}")
print(f"h = 0x{int(h):08x}")
print(f"W[0] = 0x{int(W0[0]):08x}")
print(f"K[0] = 0x{int(K[0]):08x}")
print(f"temp1 = 0x{int(temp1):08x}")
print(f"temp2 = 0x{int(temp2):08x}")


Round t = 0 internal values
a = 0x6a09e667
b = 0xbb67ae85
c = 0x3c6ef372
d = 0xa54ff53a
e = 0x510e527f
f = 0x9b05688c
g = 0x1f83d9ab
h = 0x5be0cd19
W[0] = 0x61626380
K[0] = 0x428a2f98
temp1 = 0x54da50e8
temp2 = 0x08909ae5


C:\Users\natha\AppData\Local\Temp\ipykernel_19112\2986050169.py:136: RuntimeWarning: overflow encountered in scalar add
  W[t] = np.uint32(W[t - 16] + W[t - 7] + s0 + s1)
C:\Users\natha\AppData\Local\Temp\ipykernel_19112\302794168.py:16: RuntimeWarning: overflow encountered in scalar add
  h
C:\Users\natha\AppData\Local\Temp\ipykernel_19112\302794168.py:23: RuntimeWarning: overflow encountered in scalar add
  Sigma0(a)


## Roughwork: Problem 5

In [15]:
import hashlib

# Check that "password" really gives the first hash
plain = "password"
h = hashlib.sha256(plain.encode("utf-8")).hexdigest()

print("Plaintext password:", plain)
print("Computed SHA-256  :", h)
print("Matches given hash:",
      h == "5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8")


Plaintext password: password
Computed SHA-256  : 5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8
Matches given hash: True


In [16]:
guesses = ["password", "cheese", "P@ssw0rd", "abc123"]

targets = {
    "5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8": "hash1",
    "873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34": "hash2",
    "b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342": "hash3",
}

for g in guesses:
    h = hashlib.sha256(g.encode("utf-8")).hexdigest()
    print(f"Guess {g!r} -> {h}")
    if h in targets:
        print(f"  This matches {targets[h]}")


Guess 'password' -> 5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8
  This matches hash1
Guess 'cheese' -> 873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34
  This matches hash2
Guess 'P@ssw0rd' -> b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342
  This matches hash3
Guess 'abc123' -> 6ca13d52ca70c883e0f0bb101e425a89e8624de51db2d2392593af6a84118090


Hash: 5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8
  Recovered password: 'password'
Hash: 873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34
  Recovered password: 'cheese'
Hash: b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342
  Recovered password: 'P@ssw0rd'
